In [21]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
plt.rcParams['image.cmap'] = 'gray'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:

import keras

import os
import datasets
import models
dataset="cifar10"
(x_train, y_train), (x_test, y_test), input_shape,num_classes = datasets.get_data(dataset)


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')





x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
cardinality=32
bottleneck_filters=128
initial_filters=256
lr=0.001
model = models.residual_network(input_shape,num_classes,cardinality,bottleneck_filters,initial_filters)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])
print(model.summary())
print(model.name)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1855 (Conv2D)            (None, 32, 32, 256)  7168        input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_61 (BatchNo (None, 32, 32, 256)  1024        conv2d_1855[0][0]                
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 32, 32, 256)  0           batch_normalization_61[0][0]     
__________________________________________________________________________________________________
conv2d_185

In [16]:


rotated_model = models.residual_network(input_shape,num_classes,cardinality,bottleneck_filters,initial_filters)
rotated_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])
print(rotated_model.summary())
print(rotated_model.name)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_2164 (Conv2D)            (None, 32, 32, 256)  7168        input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 32, 32, 256)  1024        conv2d_2164[0][0]                
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 32, 32, 256)  0           batch_normalization_71[0][0]     
__________________________________________________________________________________________________
conv2d_216

In [18]:
import experiment
batch_size = 32
epochs=10
rotated_epochs=epochs*2
scores=experiment.train_rotated(model,rotated_model,x_train,y_train,x_test,
                          y_test,num_classes,input_shape,batch_size,epochs,rotated_epochs)

Training model with unrotated dataset...
Epoch 1/10
1563/1562 [==============================] - 246s 157ms/step - loss: 1.0949 - acc: 0.6047 - val_loss: 1.0489 - val_acc: 0.6362
Epoch 2/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.6731 - acc: 0.7657 - val_loss: 0.6919 - val_acc: 0.7696
Epoch 3/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.5094 - acc: 0.8234 - val_loss: 0.6739 - val_acc: 0.7767
Epoch 4/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.4083 - acc: 0.8607 - val_loss: 0.6568 - val_acc: 0.7955
Epoch 5/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.3256 - acc: 0.8875 - val_loss: 0.5215 - val_acc: 0.8276
Epoch 6/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.2610 - acc: 0.9090 - val_loss: 0.4585 - val_acc: 0.8528
Epoch 7/10
1563/1562 [==============================] - 245s 157ms/step - loss: 0.2138 - acc: 0.9254 - val_loss: 0.5526 - val_acc: 

In [19]:
from datetime import datetime

output_file="results_resnet_b.txt"
f= open(output_file,"a+")
general_message="%s\nResults on dataset %s with model %s and rotated model %s\n" % (str(datetime.now()),dataset,model.name,rotated_model.name)
f.write(general_message)
print(general_message)
for k,v in scores.items():
    message='%s score: loss=%f, accuracy=%f\n' % (k,v[0],v[1])
    print(message)
    f.write(message)
f.write("\n\n")    
f.close()

2018-02-28 20:50:50.042450
Results on dataset cifar10 with model residual_network and rotated model residual_network

rotated_model_test_dataset score: loss=0.663084, accuracy=0.770700

model_test_dataset score: loss=0.570121, accuracy=0.841000

rotated_model_rotated_test_dataset score: loss=0.723364, accuracy=0.750500

model_rotated_test_dataset score: loss=4.929166, accuracy=0.279200

